In [21]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import pickle
import os
from simtk.openmm import XmlSerializer
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping


In [15]:
from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
    def generate_htf(self, factory, topology_proposal, old_positions, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex):
        htf = factory(topology_proposal=topology_proposal,
                                      current_positions=old_positions,
                                      new_positions=new_positions,
                                      use_dispersion_correction=True,
                                      functions=None,
                                      softcore_alpha=None,
                                      bond_softening_constant=1.0,
                                      angle_softening_constant=1.0,
                                      soften_only_new=False,
                                      neglected_new_angle_terms=[],
                                      neglected_old_angle_terms=[],
                                      softcore_LJ_v2=True,
                                      softcore_electrostatics=True,
                                      softcore_LJ_v2_alpha=0.85,
                                      softcore_electrostatics_alpha=0.3,
                                      softcore_sigma_Q=1.0,
                                      interpolate_old_and_new_14s=flatten_exceptions,
                                      omitted_terms=None,
                                      endstate=repartitioned_endstate,
                                      flatten_torsions=flatten_torsions)
        if is_complex:
            if factory == HybridTopologyFactory:
                self.complex_htf = htf
            elif factory == RepartitionedHybridTopologyFactory:
                if repartitioned_endstate == 0:
                    self.complex_rhtf_0 = htf
                elif repartitioned_endstate == 1:
                    self.complex_rhtf_1 = htf
        else:
            if factory == HybridTopologyFactory:
                self.apo_htf = htf
            elif factory == RepartitionedHybridTopologyFactory:
                if repartitioned_endstate == 0:
                    self.apo_rhtf_0 = htf
                elif repartitioned_endstate == 1:
                    self.apo_rhtf_1 = htf
                    
                    
    def _solvate(self,
               topology,
               positions,
               water_model,
               phase,
               ionic_strength,
               box_dimensions=None):
        """
        Generate a solvated topology, positions, and system for a given input topology and positions.
        For generating the system, the forcefield files provided in the constructor will be used.
        Parameters
        ----------
        topology : app.Topology
            Topology of the system to solvate
        positions : [n, 3] ndarray of Quantity nm
            the positions of the unsolvated system
        forcefield : SystemGenerator.forcefield
            forcefield file of solvent to add
        water_model : str
            solvent model to use for solvation
        phase : str
            if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
        ionic_strength : float * unit.molar
            the total concentration of ions (both positive and negative) to add using Modeller.
            This does not include ions that are added to neutralize the system.
            Note that only monovalent ions are currently supported.
        Returns
        -------
        solvated_topology : app.Topology
            Topology of the system with added waters
        solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
            Solvated positions
        solvated_system : openmm.System
            The parameterized system, containing a barostat if one was specified.
        """
        modeller = app.Modeller(topology, positions)
    
        geompadding = 0.9 * unit.nanometers
        maxSize = max(max((pos[i] for pos in positions))-min((pos[i] for pos in positions)) for i in range(3))
        vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
        boxVectors = [(maxSize+geompadding)*v for v in vectors]
        
        # Now we have to add missing atoms
        if phase != 'vacuum':
            _logger.info(f"solvating at {ionic_strength} using {water_model}")
            modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
#             if not box_dimensions:
#                 modeller.addSolvent(self.system_generator.forcefield, model=water_model, padding=0.9 * unit.nanometers, ionicStrength=ionic_strength)
#             else:
#                 modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxSize=box_dimensions, ionicStrength=ionic_strength)
        else:
            pass

        solvated_topology = modeller.getTopology()
#         if box_dimensions:
#             solvated_topology.setUnitCellDimensions(box_dimensions)
#         solvated_topology.setPeriodicBoxVectors(boxVectors)
        solvated_positions = modeller.getPositions()

        # Canonicalize the solvated positions: turn tuples into np.array
        solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
        solvated_system = self.system_generator.create_system(solvated_topology)

        return solvated_topology, solvated_positions, solvated_system

In [13]:
for i in range(htf.hybrid_system.getForce(8).getNumGlobalParameters()):
    print(htf.hybrid_system.getForce(8).getGlobalParameterName(i))

softcore_alpha
lambda_sterics_core
lambda_electrostatics_core
lambda_sterics_insert
lambda_sterics_delete


In [17]:
# Generate htf
solvent_delivery = PointMutationExecutor2("../../input/rbd_protonated.pdb",
                        '1', # First chain is the barstar one
                        '501',
                        'ALA',
                        ligand_input="../../input/ace2_protonated.pdb",
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml', '/home/zhangi/choderalab/openmmforcefields/amber/ffxml/GLYCAM_06j-1.xml'],
                        flatten_torsions=True,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                        )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.15 M using tip3p
INFO:setup:solvating at 0.15 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 1
INFO:geometry:Atom index propo

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.006694119787455299.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 1716 bond forces in the no-nonbonded final system
INFO:geometry:	there are 5906 angle forces in the no-nonbonded final system
INFO:geometry:	there are 11180 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 23394.256976997072


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -119.58794055741666
INFO:geometry:final reduced energy 23274.669027877113
INFO:geometry:sum of energies: 23274.669036439656
INFO:geometry:magnitude of difference in the energies: 8.562542717527322e-06
INFO:geometry:Final logp_proposal: 42.23810426310473


added energy components: [('CustomBondForce', 1.391536201473014), ('CustomAngleForce', 1.9490930168024763), ('CustomTorsionForce', 14.458137952558326), ('CustomBondForce', -137.3867077282505)]


INFO:setup:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=13.556500000000002, y=0

In [19]:
i = 0

apo_htf = solvent_delivery.get_apo_htf()
with open(os.path.join(f"{i}_apo.pickle"), "wb") as f:
    pickle.dump(apo_htf, f)

complex_htf = solvent_delivery.get_complex_htf()
with open(os.path.join(f"{i}_complex.pickle"), "wb") as f:
    pickle.dump(complex_htf, f)



NameError: name 'render_protein_residue_atom_mapping' is not defined

In [22]:
# Render atom map
atom_map_filename = f'atom_map.png'
render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

traj = md.Trajectory(apo_htf.old_positions(apo_htf.hybrid_positions), md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
traj.save(os.path.join("apo_old.pdb"))

traj = md.Trajectory(apo_htf.new_positions(apo_htf.hybrid_positions), md.Topology.from_openmm(apo_htf._topology_proposal.new_topology))
traj.save(os.path.join("apo_new.pdb"))

traj = md.Trajectory(complex_htf.old_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.old_topology))
traj.save(os.path.join("complex_old.pdb"))

traj = md.Trajectory(complex_htf.new_positions(complex_htf.hybrid_positions), md.Topology.from_openmm(complex_htf._topology_proposal.new_topology))
traj.save(os.path.join("complex_new.pdb"))

In [23]:
with open('rbd_ace2_hybrid_system.xml', 'w') as outfile:
    system_xml = XmlSerializer.serialize(complex_htf.hybrid_system)
    outfile.write(system_xml)

In [24]:
from simtk.openmm import VerletIntegrator
from simtk import openmm

In [25]:
# Check the potential energy of the nb force (total energy)
integrator = VerletIntegrator(0.001)
context = openmm.Context(complex_htf.hybrid_system, integrator)
context.setPositions(complex_htf.hybrid_positions)
state = context.getState(getPositions=True)

In [27]:
with open('rbd_ace2_state.xml', 'w') as outfile:
    state_xml = XmlSerializer.serialize(state)
    outfile.write(state_xml)

In [20]:
complex_htf.hybrid_topology.n_atoms

185378

In [29]:
complex_htf.hybrid_system.getForce(8).getUseLongRangeCorrection()

True